# 0) Test model example 

This notebook tests the different models (HAY, HAY_AIS, HAY_AIS_HILLOCK) on a simple step stimulus.

In [ ]:
import json
import sys
import os
import time
import numpy as np
from pathlib import Path

import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import MEAutility as mu

import shutil

import multimodalfitting as mf

%matplotlib widget

In [ ]:
base_dir = Path("..")

In [ ]:
probe_type = "planar"
model_name = "hay_ais" # "hay", "hay_ais", "hay_ais_hillock"
cell_models_folder = base_dir / "cell_models"

model_folder = cell_models_folder / f"{model_name}"

In [ ]:
ais_recordings = mf.utils.get_ais_extra_recordings()

In [ ]:
# instantiate cell model
cell_opt = mf.create_ground_truth_model(model_name=model_name, 
                                    release=False)
param_names = [p_name for (p_name, p) in cell_opt.params.items() if not p.frozen]

In [ ]:
param_names

In [ ]:
# instantiate cell model
cell = mf.create_ground_truth_model(model_name=model_name, 
                                    release=True)

# define a probe (this can also be defined externally)
probe = mf.define_electrode(probe_type=probe_type)

# define a test protocol
protocols = mf.define_test_step_protocol(step_amplitude=0.5, tot_duration=2000, step_duration=1500, probe=probe,
                                         extra_recordings=dict(TestStep=ais_recordings))

# instantiate simulator
sim = ephys.simulators.LFPySimulator(
    cell, cvode_active=True, electrode=probe, mechanisms_directory=model_folder)


In [ ]:
t_start = time.time()
responses = protocols["TestStep"].run(cell, param_values={}, sim=sim)
print(f"Elapsed time: {time.time() - t_start}")

In [ ]:
fig = mf.plot_responses(responses, return_fig=True)

In [ ]:
eap = mf.calculate_eap(responses, protocols=protocols, protocol_name="TestStep")

In [ ]:
max_chan, _ = np.unravel_index(np.argmin(eap), eap.shape)

In [ ]:
fig_eap, ax_eap = plt.subplots()
ax_eap = mu.plot_probe(probe, ax=ax_eap, type="planar") #, alpha_prb=0.1, alpha_elec=0.3)
ax_eap = mu.plot_mea_recording(eap, probe, colors="k", ax=ax_eap, lw=0.5)
ax_eap = mu.plot_mea_recording(eap, probe, channels=[max_chan], lw=2, colors="g", ax=ax_eap)
ax_eap.plot(*probe.positions[max_chan, :-1], color="k", marker="o")
mf.plot_cell(cell, sim=sim, color_ais="g", color_myelin="b", detailed=True, ax=ax_eap)

ax_eap.set_xlim(-100, 100)
ax_eap.set_ylim(-200, 800)